In [1]:
from flask import *
from flask_sqlalchemy import *

In [2]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'sk'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///library.db'
db = SQLAlchemy(app)
# Gets created if it doesn't exist 
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
def restart():
    global app
    global db
    app = Flask(__name__)
    app.config['SECRET_KEY'] = 'sk'
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///library.db'
    app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
    db = SQLAlchemy(app)
restart()

In [3]:
class author(db.Model):
    aid = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String, nullable=False)
    books = db.relationship('Book', backref='author', lazy=True)
    pass
class Book(db.Model):
    bid = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String, nullable=False)
    author_id = db.Column(db.Integer, 
                          db.ForeignKey('author.aid'), nullable=False)
    pass

In [4]:
@app.route('/')
def index():
    authors = author.query.all()
    return render_template('index.html', authors=authors)

In [5]:
@app.route('/addauth', methods=['POST'])
def add_author():
    name = request.form['Author']
    new_row = author(name=name)
    db.session.add(new_row)
    db.session.commit()   
    return redirect('/')

In [6]:
@app.route('/addbook/<int:aid>', methods=['POST'])
def add_book(aid):
    title = request.form['Title']
    aut = author.query.get(aid) 
    if aut:
        new_row = Book(title=title, author_id=aid)
        db.session.add(new_row)
        db.session.commit()
    return redirect('/')


In [7]:
with app.app_context():
    db.create_all()
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/May/2024 15:31:38] "GET / HTTP/1.1" 200 -
/var/folders/_k/k1v2lnnd0knd7bplvfyxt_0c0000gn/T/ipykernel_64530/2242788615.py:4: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  aut = author.query.get(aid)
127.0.0.1 - - [14/May/2024 15:37:18] "POST /addbook/1 HTTP/1.1" 302 -
127.0.0.1 - - [14/May/2024 15:37:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 15:38:21] "POST /addauth HTTP/1.1" 302 -
127.0.0.1 - - [14/May/2024 15:38:21] "GET / HTTP/1.1" 200 -
